# Brain Tumor Detection Using DL

zipfile module is used to provide tool for working on zip file

with is a keyword to open the zipfile in r (read) mode

"as zip" assigns the opened Zip file object to zip variable
extract all extracts all the contents of zip file

In [ ]:
from zipfile import ZipFile
dataset = "/content/Brain_Tumor_Dataset.zip"
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("Done")

Done


Numpy is a module with the help of which we can perform all the array operations

Matplotlib.pyplot is used for 2D plotting and vizualizing data

OS allows use functions related to working directories

Shutil helps use to copy and remove files from or to a folder

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

ROOT_DIR is used to set the path of of our dataset

images is a dictionary used to store the counts of Tumor and Healthy images

os.listdir(ROOT_DIR) returns a list of all the subdirectories present in our dataset for example here Tumor and Healthy are subdirectories

os.path.join is used to construct a full path from the dataset to the subdirectories

len function counts the number of images and stores them corresponding to respective KEY value ie dir/subdirectories

In [ ]:
ROOT_DIR='/content/Brain_Tumor_Dataset'
images={}
for dir in os.listdir(ROOT_DIR):
    images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)))

In [ ]:
images.items()

dict_items([('Tumor', 3516), ('Healthy', 1950)])

Functions of os.listdir()

In [ ]:
os.listdir("/content/Brain_Tumor_Dataset")

['Tumor', 'Healthy']

check if the train and test directory already exists

if not create both the directories using mkdir (make directories)

for each directory (test and train) make a subdirectories corresponding to the initial subdirectories(dir) ie HEALTHY and TUMOR

Now randomly choose images from the Subdirectories 1st from Healthy and then from Tumor and os.listdir(full_path) tells us from where we need to pick the data

now take 70% of the total images in a subdirectory, we did -5 so that not all the images might get moved to test/train datasets and replace is FALSE it ensures that each image is selected only once

Now O is a varibale which hold full path to a particular image and D is a variable which holds full path to our destionation directory

By using shutil.copy we copy the image from O to D

In [ ]:
if not os.path.exists("./train"):
  os.mkdir("./train")
  for dir in os.listdir(ROOT_DIR):
    os.makedirs("./train/"+dir)
    for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR,dir)),size= (math.floor(70/100*images[dir])-5),replace=False):
      O=os.path.join(ROOT_DIR,dir,img)
      D=os.path.join("./train",dir, img)  # Also add the image name to the destination path
      shutil.copy(O, D)  # Copy instead of removing
else:
  print("already exist")

if not os.path.exists("./test"):
  os.mkdir("./test")
  for dir in os.listdir(ROOT_DIR):
    os.makedirs("./test/"+dir)
    for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR,dir)),size= (math.floor(30/100*images[dir])-5),replace=False):
      O=os.path.join(ROOT_DIR,dir,img)
      D=os.path.join("./test",dir, img)  # Also add the image name to the destination path
      shutil.copy(O, D)  # Copy instead of removing
else:
  print("already exist")

In [ ]:
images={}
for dir in os.listdir(ROOT_DIR):
    images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)))
images.items()

dict_items([('Tumor', 3516), ('Healthy', 1950)])

We have inputed several Layer from Keras

CONV2D used processing and detecting/extracting features in 2-dimensional data, such as images

MaxPool2D is used to reduce the size of an image by dividing the image into a Matrix and then selecting the max values

Dropout is used to avoid overfiting

Flatten to convert 2D image to 1D array

Dense Layer where each input node is connected to each output node

BatchNormalization is used to stabalize the inputs of a neural and train them

GlobalAvgPool2D Layer is computes the avg of all the feautures and reduces dimensions to a signle valur per feature

Sequential starts building a stack of layers

 Adam is an optimization algorithm used to train deep learning models

 ImageDataGenerator used to transform and process the image through dimensions and color

In [ ]:
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAvgPool2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

32 filter each of size 3*3 , activation function relu introduces non linearity and we have an u=input shape of 224*224 with three colors RGB

Poolin window size if 2*2

Dropout layer randomly ignore 50% of neurons in previous layer to avoid overfitting

Flatten conerts 2D image to 1D vector

128 nuerons in each layer which accepts and send outputs

Last layer ie the output layer has 1 nueron

Lastly we compile our model with adam as optimizer (optimizer basically adjusts the weight of nueral networks) loss function used for Binary Classification and parameters on which the model will measure its perfromance is accuracy

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPool2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

Augments and prepares images for training by applying transformations like rescaling pixel values, shearing, zooming, and flipping

flow from directory tells us the location target size, no of images transfered at a time and binary labels of images

In [ ]:
train_images=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
train_data=train_images.flow_from_directory(directory='/content/train',target_size=(224,224),batch_size=32,class_mode='binary')

Found 3816 images belonging to 2 classes.


In [ ]:
test_images=ImageDataGenerator(rescale=1./255)
test_data=test_images.flow_from_directory(directory='/content/test',target_size=(224,224),batch_size=32,class_mode='binary')

Found 1629 images belonging to 2 classes.


EarlyStopping is used to stop training when a monitored metric has stopped improving. This helps prevent overfitting val_accuracy is used to monitor the perfromance of the model, wait for the number of epochs till no improvement

ModelCheckPoint saves the model after each epoch and saves the best model only

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
es=EarlyStopping(monitor="val_accuracy",patience=5,verbose=1,mode='min')
mc=ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.h5",verbose=1,save_best_only=True,mode='min')
cb=[es,mc]


We use fit method to train out modelwhich trains on train_data and validates the model using test data,epochs =30 means how many times the model will iterate over the dataset and callbacks is used to save the best model

In [ ]:
hs = model.fit(
    train_data,
    steps_per_epoch=len(train_data),
    epochs=30,
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=cb
)

Epoch 1/30
120/120 [==============================] - ETA: 0s - loss: 0.3412 - accuracy: 0.8483
Epoch 1: val_accuracy improved from inf to 0.94598, saving model to ./bestmodel.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


120/120 [==============================] - 500s 4s/step - loss: 0.3412 - accuracy: 0.8483 - val_loss: 0.1654 - val_accuracy: 0.9460
Epoch 2/30
120/120 [==============================] - ETA: 0s - loss: 0.2088 - accuracy: 0.9201
Epoch 2: val_accuracy improved from 0.94598 to 0.89564, saving model to ./bestmodel.h5
120/120 [==============================] - 479s 4s/step - loss: 0.2088 - accuracy: 0.9201 - val_loss: 0.2455 - val_accuracy: 0.8956
Epoch 3/30
120/120 [==============================] - ETA: 0s - loss: 0.1510 - accuracy: 0.9437
Epoch 3: val_accuracy did not improve from 0.89564
120/120 [==============================] - 478s 4s/step - loss: 0.1510 - accuracy: 0.9437 - val_loss: 0.1096 - val_accuracy: 0.9589
Epoch 4/30
120/120 [==============================] - ETA: 0s - loss: 0.1430 - accuracy: 0.9486
Epoch 4: val_accuracy did not improve from 0.89564
120/120 [==============================] - 522s 4s/step - loss: 0.1430 - accuracy: 0.9486 - val_loss: 0.0988 - val_accuracy: 0.

In [ ]:
accu=model.evaluate(test_data)[1]
print(accu)

51/51 [==============================] - 60s 1s/step - loss: 0.0743 - accuracy: 0.9730
0.972989559173584


In [ ]:
from keras.models import load_model
model=load_model("./bestmodel.h5")

In [ ]:
from keras.preprocessing.image import load_img,img_to_array
path="/content/train/Tumor/Te-glTr_0001.jpg"
image=load_img(path,target_size=(224,224))
image=img_to_array(image)
image=np.expand_dims(image,axis=0)
image=image/255
pred=model.predict(image)[0][0]
if pred > 0.5:  # Adjust the threshold here
  print("Tumor")
else:
  print("Healthy")
print(pred)

1/1 [==============================] - 0s 56ms/step
Tumor
0.9855589
